In [2]:
import os
import pandas as pd
import numpy as np
import openpyxl
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment, Border, Side
from openpyxl.drawing.image import Image
import pandas as pd
from pymysql import connect
from sqlalchemy import create_engine
from docx import Document
from docx.shared import Pt
from docx.enum.table import WD_TABLE_ALIGNMENT, WD_CELL_VERTICAL_ALIGNMENT
db=connect(
    host="localhost",
    user="Hemu",
    password="Hello@World40",
    database="middetails"
)
cur=db.cursor()
path1 = "C:\\Users\\hp\\Downloads\\II B. Tech I Sem I Mid QW Marks.xlsx"
path2 = "C:\\Users\\hp\\Downloads\\site 18 CO -PO MAPPING  with iot.xlsx"
excel_file = pd.ExcelFile(path1)
xl = pd.ExcelFile(path2)
def process_sheet(sheet_name):
    strow = 1
    enrow = 5
    cur.execute(f"DROP TABLE IF EXISTS SITE_21")
    query = f"CREATE TABLE SITE_21 (Subject_Code VARCHAR(100), course_Name VARCHAR(1000), course_outcomes VARCHAR(100000), CO_number VARCHAR(10), PO1 FLOAT, PO2 FLOAT, PO3 FLOAT, PO4 FLOAT, PO5 FLOAT, PO6 FLOAT, PO7 FLOAT, PO8 FLOAT, PO9 FLOAT, PO10 FLOAT, PO11 FLOAT, PO12 FLOAT, PSO1 FLOAT, PSO2 FLOAT, Target FLOAT)"
    cur.execute(query)
    insert_query = f"INSERT INTO SITE_21 (Subject_Code, course_Name, course_outcomes, CO_number, PO1, PO2, PO3, PO4, PO5, PO6, PO7, PO8, PO9, PO10, PO11, PO12, PSO1, PSO2, TARGET) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    while True:
        data = pd.read_excel(path2, sheet_name, skiprows=range(1, strow), nrows=enrow - strow + 1)
        
        if data.isnull().all().all():
            break
        
        data.replace('-', np.nan, inplace=True)  # Replace '-' with NaN
        data.fillna(value=np.nan, inplace=True)  # Replace 0 with NaN
        data.replace(0, np.nan, inplace=True)
        #print(data)     
        for _, row in data.iterrows():
            values = (
                row['subject code'], row['course Name'], row['course outcomes'], row['CO number'],
                row['PO1'], row['PO2'], row['PO3'], row['PO4'], row['PO5'], row['PO6'], row['PO7'],
                row['PO8'], row['PO9'], row['PO10'], row['PO11'], row['PO12'], row['PSO1'], row['PSO2'],
                row['TARGET']
            )
    # Replace NaN values with None
            values = [None if pd.isna(val) else val for val in values]
            cur.execute(insert_query, values)

        db.commit()
        
        strow = enrow + 3
        enrow = strow + 4



tbnames=[]
def analysis_sheet(sheet_name, acy, tbname):
    data = pd.read_excel(path1, sheet_name, header=[3])
    data.fillna(0, inplace=True)
    col_headers = list(data.columns)
    if 'M1Q2a' in col_headers:
        data['M1Q2a'] = data['M1Q2a'] + data['M1Q2b']
        data.rename(columns={'M1Q2a': 'M1Q2'}, inplace=True)
    if 'M1Q1a' in col_headers:
        data['M1Q1a'] = data['M1Q1a'] + data['M1Q1b']
        data.rename(columns={'M1Q1a': 'M1Q1'}, inplace=True)
    if 'M1Q3a' in col_headers:
        data['M1Q3a']=data['M1Q3a']+data['M1Q3b']
        #data.drop(columns=['M1Q3b'], inplace=True)
        data.rename(columns={'M1Q3a': 'M1Q3'}, inplace=True)
    if 'M2Q1a' in col_headers:
        data['M2Q1a']=data['M2Q1a']+data['M2Q1b']
        #data.drop(columns=['M2Q1a'], inplace=True)
        data.rename(columns={'M2Q1a': 'M2Q1'}, inplace=True)
    if 'M2Q2a' in col_headers:
        data['M2Q2a']=data['M2Q2a']+data['M2Q2b']
        #data.drop(columns=['M2Q2a'], inplace=True)
        data.rename(columns={'M2Q2a': 'M2Q2'}, inplace=True)
    if 'M2Q3a' in col_headers:
        data['M2Q3a']=data['M2Q3a']+data['M2Q3b']
        #data.drop(columns=['M2Q3a'], inplace=True)
        data.rename(columns={'M2Q3a': 'M2Q3'}, inplace=True)
    q1=data['M1Q1'][:67]
    q2=data['M1Q2'][:67]
    q3=data['M1Q3'][:67]
    n=len(q1)
    l1_q1=np.where(q1>=5)
    l1_q2=np.where(q2>=5)
    l1_q3=np.where(q3>=5)
    m1_quiz=np.squeeze(data['M1QI1'][:67])
    m1_assign=np.squeeze(data['M1A1'][:67])
    m1_assign=np.where(m1_assign>=2.5)
    m1_quiz=np.where(m1_quiz>=5)
    per_l1_q1=(len(l1_q1[0])/n)*100
    per_l1_q2=(len(l1_q2[0])/n)*100
    per_l1_q3=(len(l1_q3[0])/n)*100
    per_mid1=[per_l1_q1, per_l1_q2, per_l1_q3]
    m1_levels=[]
    for i in range(0,3):
        if (per_mid1[i]>=50) and (per_mid1[i]<65):
            m1_levels.append(1)
        elif (per_mid1[i]>=65) and (per_mid1[i]<80):
            m1_levels.append(2)
        elif (per_mid1[i]>=80) and (per_mid1[i]<=100):
            m1_levels.append(3)
        else:
            m1_levels.append(0)
    per_m1_assign=(len(m1_assign[0])/n)*100
    if per_m1_assign>=50 and per_m1_assign<65:
        m1_assign_levels=[1,1,1]
    elif per_m1_assign>=65 and per_m1_assign<80:
        m1_assign_levels=[2,2,2]
    elif per_m1_assign>=80 and per_m1_assign<=100:
        m1_assign_levels=[3,3,3]
    else:
        m1_assign_levels=[0,0,0]
    per_m1_quiz=(len(m1_quiz[0])/n)*100
    if per_m1_quiz>=50 and per_m1_quiz<65:
        m1_quiz_levels=[1,1,1]
    elif per_m1_quiz>=65 and per_m1_quiz<80:
        m1_quiz_levels=[2,2,2]
    elif per_m1_quiz>=80 and per_m1_quiz<=100:
        m1_quiz_levels=[3,3,3]
    else:
        m1_quiz_levels=[0,0,0]
    q1=data['M2Q1'][:67]
    q2=data['M2Q2'][:67]
    q3=data['M2Q3'][:67]
    m2_q1=np.where(q1>=5)
    m2_q2=np.where(q2>=5)
    m2_q3=np.where(q3>=5)
    m2_quiz=np.squeeze(data['M2QI2'][:67])
    m2_assign=np.squeeze(data['M2A2'][:67])
    m2_assign=np.where(m2_assign>=2.5)
    m2_quiz=np.where(m2_quiz>=5)
    per_m2_q1=(len(m2_q1[0])/n)*100
    per_m2_q2=(len(m2_q2[0])/n)*100
    per_m2_q3=(len(m2_q3[0])/n)*100
    per_m2=[per_m2_q1, per_m2_q1, per_m2_q1]
    m2_levels=[]
    for i in range(0,3):
        if (per_m2[i]>=50) and (per_m2[i]<65):
            m2_levels.append(1)
        elif (per_m2[i]>=65) and (per_m2[i]<80):
            m2_levels.append(2)
        elif (per_m2[i]>=80) and (per_m2[i]<=100):
            m2_levels.append(3)
        else:
            m2_levels.append(0)
    per_m2_assign=(len(m2_assign[0])/n)*100
    if per_m2_assign>=50 and per_m2_assign<65:
        m2_assign_levels=[1,1,1]
    elif per_m2_assign>=65 and per_m2_assign<80:
        m2_assign_levels=[2,2,2]
    elif per_m2_assign>=80 and per_m2_assign<=100:
        m2_assign_levels=[3,3,3]
    else:
        m2_assign_levlels=[0,0,0]
    per_m2_quiz=(len(m2_quiz[0])/n)*100
    
    if per_m2_quiz>=50 and per_m2_quiz<65:
        m2_quiz_levels=[1,1,1]
    elif per_m2_quiz>=65 and per_m2_quiz<80:
        m2_quiz_levels=[2,2,2]
    elif per_m1_quiz>=80 and per_m2_quiz<=100:
        m2_quiz_levels=[3,3,3]
    else:
        m2_quiz_levels=[0,0,0]
    mid_levels=m1_levels+m2_levels
    mid_levels.pop(2)
    mid_levels[2]=max(m1_levels[2], m2_levels[0])
    CO=['CO1','CO2','CO3', 'CO4', 'CO5']
    assign_levels=m1_assign_levels+m2_assign_levels
    quiz_levels=m1_quiz_levels+m2_quiz_levels
    assign_levels.pop(2)
    quiz_levels.pop(2)
    assign_levels[2]=max(m1_assign_levels[2], m2_assign_levels[0])
    quiz_levels[2]=max(m1_quiz_levels[2], m2_quiz_levels[0])
    COAF=[1,3, 2, 2, 3]
    eq1=data['Q1'][:67]
    eq2=data['Q2'][:67]
    eq3=data['Q3'][:67]
    eq4=data['Q4'][:67]
    eq5=data['Q5'][:67]
    eq1=np.where(eq1>=5)
    eq2=np.where(eq2>=5)
    eq3=np.where(eq3>=5)
    eq4=np.where(eq4>=5)
    eq5=np.where(eq5>=5)
    e_levels=[]
    per_e_q1=(len(eq1[0])/n)*100
    per_e_q2=(len(eq2[0])/n)*100
    per_e_q3=(len(eq3[0])/n)*100
    per_e_q4=(len(eq4[0])/n)*100
    per_e_q5=(len(eq5[0])/n)*100
    per_e=[per_e_q1, per_e_q2, per_e_q3, per_e_q4, per_e_q5]
    for i in range(0,5):
        if (per_e[i]>=50) and (per_e[i]<65):
            e_levels.append(1)
        elif (per_e[i]>=65) and (per_e[i]<80):
            e_levels.append(2)
        elif (per_e[i]>=80) and (per_e[i]<=100):
            e_levels.append(3)
        else:
            e_levels.append(0)
    avg=[]
    for i in range(0, 5):
        avg.append((mid_levels[i]+quiz_levels[i]+assign_levels[i]+COAF[i]+e_levels[i])/5)
    cur.execute(f"SELECT target FROM site_21 where subject_code='{sheet_name}'")
    target=cur.fetchall()
    target = [item[0] for item in target]
    sns=[sheet_name, sheet_name, sheet_name, sheet_name, sheet_name]
    insert_query = f"INSERT INTO {tbname} (Academic_Year, Subject_Code, Course_Outcome, MID, Assignment, Quiz, Externals, COAF, Average, Target) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    values = zip(acy, sns, CO, mid_levels, assign_levels, quiz_levels, e_levels, COAF, avg, target)
    #tbnames+=tbname
   # print(tbname)
   # for i in values:
     #   print(i, end=" ")
    #print()
    for value in values:
        cur.execute(insert_query, value)
        #print(value)
        db.commit()
       # print(value)
   # print()
def doc_process(tbname):
    doc = Document()
    for sn in sheet_names1:
        cur.execute(f"SELECT * FROM {tbname} where subject_code='{sn}'")
        data = cur.fetchall()
        column_names = [desc[0] for desc in cur.description]
        doc.add_heading(sn, level=1)
        table = doc.add_table(rows=1, cols=len(column_names))
        table.alignment = WD_TABLE_ALIGNMENT.CENTER
        table.style = 'Table Grid'
        hdr_cells = table.rows[0].cells
        for i, column_name in enumerate(column_names):
            hdr_cells[i].text = column_name
            hdr_cells[i].paragraphs[0].runs[0].font.bold = True
        for row in data:
            row_cells = table.add_row().cells
            for i, value in enumerate(row):
                row_cells[i].text = str(value)

    doc.save("output.docx")
    
def po_assessment(sheet_name, tabname, sub, name):
    wb = openpyxl.load_workbook(name)
# Query the database
    query = f"SELECT PO1, PO2, PO3, PO4, PO5, PO6, PO7, PO8, PO9, PO10, PO11, PO12, PSO1, PSO2 FROM site_21 WHERE subject_code='{sheet_name}'"
    cur.execute(query)
    data = cur.fetchall()

    # Fetch course name
    query = f"SELECT course_name FROM site_21 WHERE subject_code='{sheet_name}'"
    cur.execute(query)
    subnames = [item[0] for item in cur.fetchall()]

    # Fetch averages

    query = f"SELECT average FROM `{tabname}` WHERE subject_code='{sheet_name}'"
    cur.execute(query)
    avgs = [item[0] for item in cur.fetchall()]
    avg = sum(avgs) / len(avgs)  # Calculate the average of avgs

    # Create a workbook and worksheet
    
    ws = wb.create_sheet(subnames[0])

    # Define the border style
    thin_border = Border(
        left=Side(style='thin'),
        right=Side(style='thin'),
        top=Side(style='thin'),
        bottom=Side(style='thin')
    )

    # Define header font style
    header_font = Font(name='Times New Roman', size=12, bold=True)
    value_font = Font(name='Times New Roman', size=12)
    # Merging cells and setting alignment
    ws.merge_cells('A1:H1')
    ws.merge_cells('I1:Q1')
    for row in ws['A1:H1']:
        for cell in row:
            cell.border = thin_border
            cell.alignment = Alignment(horizontal='center')
            cell.font = header_font
    ws['A1'] = 'Merged Cells'

    for row in ws['I1:Q1']:
        for cell in row:
            cell.border = thin_border
            cell.alignment = Alignment(horizontal='center')
            cell.font = header_font
    ws['I1'] = 'Course Name: ' + sheet_name

    ws.merge_cells('A2:A3')
    for row in ws['A2:A3']:
        for cell in row:
            cell.border = thin_border
            cell.alignment = Alignment(vertical='center')
            cell.font = header_font
    ws['A2'] = 'Subject Name'

    ws.merge_cells('B2:B3')
    for row in ws['B2:B3']:
        for cell in row:
            cell.border = thin_border
            cell.alignment = Alignment(vertical='center')
            cell.font = header_font
    ws['B2'] = 'course Name'

    ws.merge_cells('C2:N2')
    for row in ws['C2:N2']:
        for cell in row:
            cell.border = thin_border
            cell.alignment = Alignment(horizontal='center')
            cell.font = header_font
    ws['C2'] = 'PO'

    ws.merge_cells('O2:P2')
    for row in ws['O2:P2']:
        for cell in row:
            cell.border = thin_border
            cell.alignment = Alignment(horizontal='center')
            cell.font = header_font
    ws['O2'] = 'PSO'

    ws.merge_cells('Q2:Q3')
    for row in ws['Q2:Q3']:
        for cell in row:
            cell.border = thin_border
            cell.alignment = Alignment(vertical='center')
            cell.font = header_font
    ws['Q2'] = 'CO Target'

    ws.merge_cells('A4:A8')
    for row in ws['A4:A8']:
        for cell in row:
            cell.border = thin_border
            cell.alignment = Alignment(vertical='center')
            cell.font = header_font
    ws['A4'] = subnames[0]

    ws.merge_cells('A9:B9')
    for row in ws['A9:B9']:
        for cell in row:
            cell.border = thin_border
            cell.alignment = Alignment(horizontal='center')
            cell.font = header_font
    ws['A9'] = 'Total'

    ws.merge_cells('A11:C11')
    for row in ws['A11:C11']:
        for cell in row:
            cell.border = thin_border
            cell.alignment = Alignment(horizontal='center')
            cell.font = header_font
    ws['A11'] = 'Average Level (Direct)'

    ws.merge_cells('A12:A16')
    for row in ws['A12:A16']:
        for cell in row:
            cell.border = thin_border
            cell.alignment = Alignment(vertical='center')
            cell.font = header_font
    ws['A12'] = subnames[0]

    ws.merge_cells('A17:B17')
    for row in ws['A17:B17']:
        for cell in row:
            cell.border = thin_border
            cell.alignment = Alignment(horizontal='center')
            cell.font = header_font
    ws['A17'] = 'CO Attainment'

    ws.merge_cells('C19:N19')
    for row in ws['C19:N19']:
        for cell in row:
            cell.border = thin_border
            cell.alignment = Alignment(horizontal='center')
            cell.font = header_font
    ws['C19'] = 'PO'

    ws.merge_cells('O19:P19')
    for row in ws['O19:P19']:
        for cell in row:
            cell.border = thin_border
            cell.alignment = Alignment(horizontal='center')
            cell.font = header_font
    ws['O19'] = 'PSO'

    ws.merge_cells('A21:A25')
    for row in ws['A21:A25']:
        for cell in row:
            cell.border = thin_border
            cell.alignment = Alignment(vertical='center')
            cell.font = header_font
    ws['A21'] = subnames[0]
    ws.merge_cells('A26:B26')
    for row in ws['A26:B26']:
        for cell in row:
            cell.border = thin_border
            cell.alignment = Alignment(horizontal='center')
            cell.font = header_font
    ws['A26'] = 'Total'
    ws.merge_cells('A27:B27')
    for row in ws['A27:B27']:
        for cell in row:
            cell.border = thin_border
            cell.alignment = Alignment(horizontal='center')
            cell.font = header_font
    ws['A27'] = 'CO Level Total'

    ws.merge_cells('A28:B28')
    for row in ws['A28:B28']:
        for cell in row:
            cell.border = thin_border
            cell.alignment = Alignment(horizontal='center')
            cell.font = header_font
    ws['A28'] = 'PO Level'

    # Setting PO values
    PO = ['C21' + str(sub) + '.1', 'C21' + str(sub) + '.2', 'C21' + str(sub) + '.3', 'C21' + str(sub) + '.4', 'C21' + str(sub) + '.5']
    for i in range(4, 9):
        ws.cell(row=i, column=2, value=PO[i - 4]).border = thin_border
        ws.cell(row=i, column=2).alignment = Alignment(horizontal='center')
        ws.cell(row=i, column=2).font = value_font
        ws.cell(row=i + 8, column=2, value=PO[i - 4]).border = thin_border
        ws.cell(row=i + 17, column=2, value=PO[i - 4]).border = thin_border

    # Setting CO values
    s = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2]
    for col_index, value in enumerate(s, start=3):
        ws.cell(row=3, column=col_index, value=value).border = thin_border
        ws.cell(row=20, column=col_index, value=value).border = thin_border

    # Check if data is not empty before calculating column sums
    if data:
        column_sums1 = []
        column_sums2 = []
        #i=
        for col_index in range(len(data[0])):
            column_values1 = [row[col_index] for row in data]
            column_values2 = [row[col_index] for row in data]
            if all(value is None for value in column_values1):
                column_sums1.append(None)
                column_sums2.append(None)
            else:
                column_sum1 = sum(value for value in column_values1 if value is not None)
                column_sum2 = sum(value * avgs[i] for i, value in enumerate(column_values2) if value is not None)
                column_sums1.append(column_sum1)
                column_sums2.append(column_sum2)
            #i+=1
        print(column_sums1)
        print(column_sums2)
        # Adding data to worksheet
        ri = 4
        co_avg = []
        for i in range(len(data)):
            for col_index, value in enumerate(data[i], start=3):
                cell = ws.cell(row=ri, column=col_index, value=value)
                cell.alignment = Alignment(horizontal='center')
                cell.font = value_font
                cell.border = thin_border
                if value is not None:
                    value = value * avgs[i]
                cell=ws.cell(row=ri + 17, column=col_index, value=value)
                cell.border = thin_border
                cell.alignment = Alignment(horizontal='center')
                cell.font = value_font
            filtered_values = [value for value in data[i] if value is not None]
            total_sum = sum(filtered_values)
            count = len(filtered_values)
            if count > 0:
                average = total_sum / count
                co_avg.append(average)
                cell=ws.cell(row=ri, column=col_index + 1, value=average)
                cell.border = thin_border
                cell.alignment = Alignment(horizontal='center')
                cell.font = value_font
            ri += 1

        # Adding column sums to worksheet
        tot = []
        for col_index, value in enumerate(column_sums1, start=3):
            cell=ws.cell(row=9, column=col_index, value=value)
            cell.border = thin_border
            cell.alignment = Alignment(horizontal='center')
            cell.font = value_font
        for col_index, value in enumerate(column_sums2, start=3):
            cell=ws.cell(row=26, column=col_index, value=value)
            cell.border = thin_border
            cell.alignment = Alignment(horizontal='center')
            cell.font = value_font
        for i in range(12, 17):
            cell=ws.cell(row=i, column=3, value=avgs[i - 12])
            cell.border = thin_border
            cell.alignment = Alignment(horizontal='center')
            cell.font = value_font
        cell=ws.cell(row=17, column=3, value=avg)
        cell.border = thin_border
        cell.alignment = Alignment(horizontal='center')
        cell.font = value_font

        # Adding final values
        cell=ws.cell(row=9, column=col_index + 1, value=sum(co_avg) / len(co_avg))
        cell.border = thin_border
        cell.alignment = Alignment(horizontal='center')
        cell.font = value_font
    
        i = 0
        for col_index, value in enumerate(column_sums1, start=3):
            cell=ws.cell(row=27, column=col_index, value=value)
            cell.border = thin_border
            cell.alignment = Alignment(horizontal='center')
            cell.font = value_font
        for i in range(14):
            if column_sums2[i] is not None:
                
                column_sums2[i]=column_sums2[i]/column_sums1[i] 
        print(column_sums2)
        for i in range(len(column_sums2)):   
            cell=ws.cell(row=28, column=i+3, value=column_sums2[i])
            cell.border = thin_border
            cell.alignment = Alignment(horizontal='center')
            cell.font = value_font
        
    # Save the workbook
    wb.save(name)
    return column_sums2, avgs, subnames[0]
def po_asses_fun(sheet_names, tbname):
    img_path = "C:\\Users\\hp\\Downloads\\DEPT LOGO.jpg"  # Replace with your image path
    img = Image(img_path)
    sub=1
    name1='PO Assessment from CO@2-1_'+ay+'.xlsx'
    name2='CO_Outcome_Direct_Attainment@2-1_'+ay+'.xlsx'
    thin_border = Border(
        left=Side(style='thin'),
        right=Side(style='thin'),
        top=Side(style='thin'),
        bottom=Side(style='thin')
    )
    header_font = Font(name='Times New Roman', size=12, bold=True)
    value_font = Font(name='Times New Roman', size=12)
    wb=Workbook()
    wb.save(name1)
    wb1=Workbook()
    ws = wb1.create_sheet("2-1")
    po_attain=[]
    ws.merge_cells('A1:O1')
    for row in ws['A1:O1']:
        for cell in row:
            
            cell.alignment = Alignment(horizontal='center')
            
    ws.add_image(img, 'A1')
    ws.merge_cells('A4:A5')
    for row in ws['A4:A5']:
        for cell in row:
            cell.border = thin_border
            cell.alignment = Alignment(vertical='center')
            cell.font = header_font
    ws['A4'] = 'Year & Sem'
    ws.merge_cells('A19:A20')
    for row in ws['A19:A20']:
        for cell in row:
            cell.border = thin_border
            cell.alignment = Alignment(vertical='center')
            cell.font = header_font
    ws['A19'] = 'Year & Sem'
    ws.merge_cells('B4:B5')
    for row in ws['B4:B5']:
        for cell in row:
            cell.border = thin_border
            cell.alignment = Alignment(vertical='center')
            cell.font = header_font
    ws['B4'] = 'Course Code'
    ws.merge_cells('B19:B20')
    for row in ws['B19:B20']:
        for cell in row:
            cell.border = thin_border
            cell.alignment = Alignment(vertical='center')
            cell.font = header_font
    ws['B19'] = 'Course Code'
    ws.merge_cells('C4:C5')
    for row in ws['C4:C5']:
        for cell in row:
            cell.border = thin_border
            cell.alignment = Alignment(vertical='center')
            cell.font = header_font
    ws['C4'] = 'Course'
    ws.merge_cells('C19:C20')
    for row in ws['C19:C20']:
        for cell in row:
            cell.border = thin_border
            cell.alignment = Alignment(vertical='center')
            cell.font = header_font
    ws['C19'] = 'Course'
    ws.merge_cells('D4:H4')
    for row in ws['D4:H4']:
        for cell in row:
            cell.border = thin_border
            cell.alignment = Alignment(horizontal='center')
            cell.font = header_font
    ws['D4'] = 'Course\n Attainment'
    ws.merge_cells('D19:U19')
    for row in ws['D19:U19']:
        for cell in row:
            cell.border = thin_border
            cell.alignment = Alignment(horizontal='center')
            cell.font = header_font
    ws['D19'] = 'POs'
    ws.merge_cells('I4:I5')
    for row in ws['I4:I5']:
        for cell in row:
            cell.border = thin_border
            cell.alignment = Alignment(vertical='center')
            cell.font = header_font
    ws['I4'] = 'Direct Course\nAttainment'
    for i in range(4, 9):
        ws.cell(row=5, column=i, value=('CO'+str(i-3))).border = thin_border
        ws.cell(row=5, column=i).alignment = Alignment(horizontal='center')
        ws.cell(row=5, column=i).font = value_font
    for i in range(4, 16):
        ws.cell(row=20, column=i, value=i-3).border = thin_border
        ws.cell(row=20, column=i).alignment = Alignment(horizontal='center')
        ws.cell(row=20, column=i).font = value_font
    
    
    row=6
    for sheet_name in sheet_names:
        po_attain, avgs, subname = po_assessment(sheet_name, tbname, sub, name1)
        cell=ws.cell(row=row, column=2, value=('C21'+str(sub)))
        cell.border=thin_border
        cell.alignment = Alignment(horizontal='center')
        cell.font=value_font
        cell=ws.cell(row=row+15, column=2, value=('C21'+str(sub)))
        cell.border=thin_border
        cell.alignment = Alignment(horizontal='center')
        cell.font=value_font
        cell=ws.cell(row=row, column=3, value=subname)
        cell.border=thin_border
        cell.alignment = Alignment(horizontal='center')
        cell.font=value_font
        cell=ws.cell(row=row+15, column=3, value=subname)
        cell.border=thin_border
        cell.alignment = Alignment(horizontal='center')
        cell.font=value_font
        for i in range(4, 9):
            cell=ws.cell(row=row, column=i, value=avgs[i-4])
            cell.border=thin_border
            cell.alignment = Alignment(horizontal='center')
            cell.font=value_font
        cell=ws.cell(row=row, column=9, value=sum(avgs)/len(avgs))
        cell.border=thin_border
        cell.alignment = Alignment(horizontal='center')
        cell.font=value_font
        for i in range(12):
            cell=ws.cell(row=row+15, column=i+4, value=po_attain[i])
            cell.border=thin_border
            cell.alignment = Alignment(horizontal='center')
            cell.font=value_font
        sub+=1
        row+=1
    ws.merge_cells('A6:A11')
    for row in ws['A6:A11']:
        for cell in row:
            cell.border = thin_border
            cell.alignment = Alignment(vertical='center')
            cell.font = header_font
    ws['A6'] = 'II B. Tech\nI sem'
    ws.merge_cells('A21:A26')
    for row in ws['A21:A26']:
        for cell in row:
            cell.border = thin_border
            cell.alignment = Alignment(vertical='center')
            cell.font = header_font
    ws['A21'] = 'II B. Tech\nI sem' 
    wb1.save(name2)
    

sheet_names1=excel_file.sheet_names
sheet_names2=xl.sheet_names
for sheet_name in sheet_names2:
    process_sheet(sheet_name)
ay="2022_2023"
batch='2021_25'
tbname="Batch_"+batch
acy=[ay, ay, ay, ay, ay]
cur.execute(f"DROP TABLE IF EXISTS {tbname}")
query = f"CREATE TABLE {tbname} (Academic_Year varchar(20), Subject_Code varchar(15), Course_Outcome VARCHAR(100), MID FLOAT, Assignment FLOAT, Quiz FLOAT, Externals FLOAT, COAF FLOAT, Average FLOAT, Target FLOAT)"
cur.execute(query)
for sheet_name in sheet_names1:
    analysis_sheet(sheet_name, acy, tbname)

po_asses_fun(sheet_names1, tbname)
doc_process(tbname)
print("Hammaya Ayipoyindhi")
cur.close()
db.close()

C:\Users\hp\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


[15.0, 15.0, None, None, None, None, None, None, None, None, None, None, None, None]
[39.0, 39.0, None, None, None, None, None, None, None, None, None, None, None, None]
[2.6, 2.6, None, None, None, None, None, None, None, None, None, None, None, None]
[15.0, 11.0, None, 2.0, None, None, None, None, None, None, None, None, 15.0, None]
[30.0, 22.2, None, 4.0, None, None, None, None, None, None, None, None, 30.0, None]
[2.0, 2.018181818181818, None, 2.0, None, None, None, None, None, None, None, None, 2.0, None]
[15.0, 13.0, 1.0, None, None, None, None, None, None, None, None, None, 2.0, 2.0]
[37.800000000000004, 32.6, 2.6, None, None, None, None, None, None, None, None, None, 5.2, 5.2]
[2.5200000000000005, 2.5076923076923077, 2.6, None, None, None, None, None, None, None, None, None, 2.6, 2.6]
[10.0, 15.0, None, None, None, None, None, None, None, None, None, None, None, None]
[25.2, 37.800000000000004, None, None, None, None, None, None, None, None, None, None, None, None]
[2.52, 2.520

In [3]:
import openpyxl
from openpyxl.styles import Font, Border, Fill, Alignment, Protection, NamedStyle
from copy import copy  # Import the copy function

def copy_styles(source_cell, target_cell):
    if source_cell.has_style:
        target_cell.font = copy(source_cell.font)
        target_cell.border = copy(source_cell.border)
        target_cell.fill = copy(source_cell.fill)
        target_cell.number_format = source_cell.number_format
        target_cell.protection = copy(source_cell.protection)
        target_cell.alignment = copy(source_cell.alignment)

def transfer_data(source_path, destination_path):
    try:
        # Load the corrupted workbook
        source_wb = openpyxl.load_workbook(source_path, data_only=False)
        
        # Create a new workbook
        destination_wb = openpyxl.Workbook()
        destination_wb.remove(destination_wb.active)  # Remove the default sheet

        # Transfer data from the source workbook to the destination workbook
        for sheet_name in source_wb.sheetnames:
            source_sheet = source_wb[sheet_name]
            new_sheet = destination_wb.create_sheet(title=sheet_name)
            
            for row in source_sheet.iter_rows():
                for cell in row:
                    new_cell = new_sheet[cell.coordinate]
                    new_cell.value = cell.value
                    copy_styles(cell, new_cell)

            # Copy column widths
            for col in source_sheet.column_dimensions:
                new_sheet.column_dimensions[col].width = source_sheet.column_dimensions[col].width

            # Copy row heights
            for row in source_sheet.row_dimensions:
                new_sheet.row_dimensions[row].height = source_sheet.row_dimensions[row].height

        # Save the new workbook
        destination_wb.save(destination_path)
        print(f"Data transferred successfully to {destination_path}")

    except Exception as e:
        print(f"An error occurred: {e}")

# Define the source (corrupted) and destination (new) file paths
source_path = 'C:\\Users\\hp\\PO Assessment@2-1_2022_2023.xlsx'  # Replace with your actual corrupted file path
destination_path = 'C:\\Users\\hp\\PO_Assessment_Recovery.xlsx'  # Replace with your desired new file path

# Transfer data from the corrupted workbook to the new workbook
transfer_data(source_path, destination_path)


Data transferred successfully to C:\Users\hp\PO_Assessment_Recovery.xlsx


In [6]:
chr(ord('d')+12)

'p'

In [8]:
a = "Hi \n Hello"
a


'Hi \n Hello'